## Подготовка данных с визитами

Для того, чтобы построить аналитику цепочек потребуется таблица с визитами из коннектора. (если нет коннектора, то можно как-то предподготовить данные из LogsAPI).

Требуемые поля таблицы:
```
CounterID, UserIDHash, VisitID, VisitVersion, Sign, 
StartDate, UTCStartTime, 
`TrafficSource.Model`, `TrafficSource.StartTime`,  
`TrafficSource.ID`, `TrafficSource.SearchEngineID`, 
`TrafficSource.AdvEngineID`, `TrafficSource.SocialSourceNetworkID`, 
`TrafficSource.ClickBannerID`, `TrafficSource.ClickTargetType`, 
`TrafficSource.RecommendationSystemID`, `TrafficSource.MessengerID`, 
`EPurchase.ID`, `EPurchase.Revenue`,
`Goals.ID`, `Goals.Price`
```

Если у вас уже подготовлена такая таблица, то шаг можно пропустить и перейти к Шагу 2 в notebook с названием `2. analysis of channels chains.ipynb`

Если таблицы нет, то можно ее создать. Например, ниже можно использовать файл с выгруженными из CH данными фейкового счетчика для примера. Данные можно скачать здесь - https://disk.yandex.ru/d/zSmOl5X_gGDGyA

Сначала инициализируем доступ к clickhouse

In [1]:
#----------Вводить свои данные в нижние 3 переменные--------------
CH_HOST_NAME = 'rc1a-4dk99mincol3hoff.mdb.yandexcloud.net'
CH_USER      = 'admin'
CH_DB_NAME   = 'metrica_data'

#-------------------------------------------
CH_PASS      = open('../_chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

In [3]:
import some_funcs
from some_funcs import simple_ch_client

In [4]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

In [5]:
my_client.get_version()

23.8.12.13



Создаем новую табличку с полями "а-ля" в коннекторе

In [12]:
drop_visits_raw_table = f'drop table if exists {CH_DB_NAME}.visits_from_ch'
my_client.get_clickhouse_data(drop_visits_raw_table)

create_visits_raw_table = f'''
CREATE TABLE {CH_DB_NAME}.visits_from_ch (
    `CounterID` UInt32, `UserIDHash` UInt64, `VisitID` UInt64, VisitVersion UInt32, Sign Int8, 
    `StartDate` Date, `UTCStartTime` DateTime, Duration UInt32,
    `TrafficSource.Model` Array(UInt8), `TrafficSource.StartTime` Array(DateTime),  
    `TrafficSource.ID` Array(Int8), `TrafficSource.SearchEngineID` Array(UInt16), 
    `TrafficSource.AdvEngineID` Array(UInt8), `TrafficSource.SocialSourceNetworkID` Array(UInt8), 
    `TrafficSource.ClickBannerID` Array(UInt64), `TrafficSource.ClickTargetType` Array(UInt16), 
    `TrafficSource.RecommendationSystemID` Array(UInt8), `TrafficSource.MessengerID` Array(UInt8), 
    `EPurchase.ID` Array(String), `EPurchase.Revenue` Array(Int64),
    `Goals.ID` Array(UInt32), `Goals.Price` Array(Int64)
) 
ENGINE = MergeTree 
ORDER BY intHash32(UserIDHash) 
SAMPLE BY intHash32(UserIDHash)
'''
my_client.get_clickhouse_data(create_visits_raw_table)

''

Читаем файл с фейковыми данными

In [18]:
import gzip, json
with gzip.open('../visits_fake_data_for_meetup_v2.tar.gz', 'rb') as fin:
    json_bytes = fin.read()  
    
json_str = json_bytes.decode('utf-8')

Загружаем в созданную таблицу

In [19]:
import requests
content = json_str.encode('utf-8')
query_dict = {
        'query': f'INSERT INTO {CH_DB_NAME}.visits_from_ch FORMAT JSON',
        'user': CH_USER, 
        'password':CH_PASS
    }
r = requests.post(CH_HOST, data=content, params=query_dict, verify=CH_CASERT)
result = r.text


Если все хорошо, то в `result` должна быть пустота

In [20]:
result

''